# Create an Agent for Amazon Bedrock integrated with Knowledge Bases and attach Action Group

This notebook involves creating an Amazon Bedrock Agent that makes use of Knowledge Bases for Amazon Bedrock to retrieve data about restaurant's menus. Purpose is to **give information to clients about the adults/childrens menus** and be in charge of the **table booking system**. Client's will be able to create, delete or get booking information. The architecture looks as following:

The steps to complete this notebook are:

1. Import the needed libraries
1. Create the Knowledge Base for Amazon Bedrock
1. Upload the dataset to Amazon S3
1. Create the Agent for Amazon Bedrock
1. Test the Agent
1. Clean-up the resources created

## 1. Import the needed libraries

First step: install the pre-requisites packages

In [2]:
!pip install --upgrade -q -r requirements.txt

In [4]:
import os
import time
import boto3
import logging
import pprint
import json

from knowledge_base import BedrockKnowledgeBase
from agent import create_agent_role_and_policies, create_lambda_role, delete_agent_roles_and_policies
from agent import create_dynamodb, create_lambda, clean_up_resources

In [5]:
# Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '346585724397')

In [6]:
suffix = f"{region}-{account_id}"
agent_name = 'booking-agent'
knowledge_base_name = f'{agent_name}-kb'
knowledge_base_description = "Knowledge Base containing the restaurant menu's collection"
agent_alias_name = "booking-agent-alias"
bucket_name = f'{agent_name}-{suffix}'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

agent_description = "Agent in charge of a restaurants table bookings"
agent_instruction = """
You are a restaurant agent, helping clients retrieve information from their booking, 
create a new booking or delete an existing booking
"""

agent_action_group_description = """
Actions for getting table booking information, create a new booking or delete an existing booking"""

agent_action_group_name = "TableBookingsActionGroup"

## 2. Create Knowledge Base for Amazon Bedrock
Let's start by creating a [Knowledge Base for Amazon Bedrock](https://aws.amazon.com/bedrock/knowledge-bases/) to store the restaurant menus. Knowledge Bases allow us to integrate with different vector databases including [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/) and [Pinecone](http://app.pinecone.io/bedrock-integration). For this example, we'll integrate the knowledge base with **Amazon OpenSearch Serverless**. To do so, we will use the helper class `BedrockKnowledgeBase` which will create the knowledge base and all of its pre-requisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

In [7]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,
    kb_description=knowledge_base_description,
    data_bucket_name=bucket_name
)

Step 1 - Creating or retrieving booking-agent-us-west-2-346585724397 S3 bucket for Knowledge Base documents
Creating bucket booking-agent-us-west-2-346585724397
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_3465) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '315',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Wed, 10 Jul 2024 13:46:51 '
                                                 'GMT',
                                         'x-amzn-requestid': '7288e2b3-4c02-44ad-8f1f-aec3ed944382'},
                        'HTTPStatusCode': 200,
                        'RequestId': '7288e2b3-4c02-44ad-8f1f-aec3ed94438

[2024-07-10 13:55:25,129] p73 {base.py:258} INFO - PUT https://ltrrfl22o4hpqb5fz0e5.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-3465 [status:200 request:1.397s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-3465',
  'shards_acknowledged': True}
Step 6 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2024, 7, 10, 13, 56, 25, 288071, tzinfo=tzlocal()),
  'description': "Knowledge Base containing the restaurant menu's collection",
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:346585724397:knowledge-base/MKEZTICEKV',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1'}},
  'knowledgeBaseId': 'MKEZTICEKV',
  'name': 'booking-agent-kb',
  'roleArn': 'arn:aws:iam::346585724397:role/AmazonBedrockExecutionRoleForKnowledgeBase_3465',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-west-2:346585724397:collection/ltrrfl22o4hpqb5fz0e5',
                                     

## 3. Upload the dataset to Amazon S3
Let's populate the knowledge base with the menu's dataset. The Knowledge Base data source expects the data to be available on the S3 Bucket connected to it and changes on the data can be syncronized to the knowledge base using the `StartIngestionJob` API call. In this example, we use the [boto3 abstraction](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html) of the API, via our helper classe. 

Let's first upload the menu's data available on the `dataset` folder to s3

In [8]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("dataset", bucket_name)

uploading file dataset/Restaurant_Childrens_Menu.pdf to booking-agent-us-west-2-346585724397
uploading file dataset/Restaurant_Dinner_Menu.pdf to booking-agent-us-west-2-346585724397
uploading file dataset/Restaurant_week_specials.pdf to booking-agent-us-west-2-346585724397


Now, start the ingestion job

In [9]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

{ 'dataSourceId': 'JPHZCNUZ5J',
  'ingestionJobId': '3AW1B6PEJJ',
  'knowledgeBaseId': 'MKEZTICEKV',
  'startedAt': datetime.datetime(2024, 7, 10, 14, 2, 35, 200571, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 7, 10, 14, 2, 35, 200571, tzinfo=tzlocal())}
{ 'dataSourceId': 'JPHZCNUZ5J',
  'ingestionJobId': '3AW1B6PEJJ',
  'knowledgeBaseId': 'MKEZTICEKV',
  'startedAt': datetime.datetime(2024, 7, 10, 14, 2, 35, 200571, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
     

Finally we collect the Knowledge Base Id to integrate it with our Agent later on

In [10]:
kb_id = knowledge_base.get_knowledge_base_id()

'MKEZTICEKV'


### 3.1 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a LLM model from Bedrock

In [11]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which are the 5 mains available in the childrens menu?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, agent_foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The 5 mains available in the children's menu are:

1. Mini Cheeseburgers
2. Fish Sticks
3. Grilled Cheese Sandwich
4. Spaghetti with Marinara Sauce
5. Mini Pita Pizza



With the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

#### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chunks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata

In [12]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': 'Which are the 5 mains available in the childrens menu?'
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Chunk 1:  The Regrettable Experience — Children's Menu Entrees:   1. CHICKEN NUGGETS   ●   ●   ●   Description: Crispy chicken nuggets served with a side of ketchup or ranch dressing.   Allergens: Gluten (in the coating), possible Soy.   Suitable for Vegetarians: No   2. MACARONI AND CHEESE   ●   ●   ●   Description: Classic macaroni pasta smothered in creamy cheese sauce.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   3. MINI CHEESE QUESADILLAS   ●   ●   ●   Description: Small flour tortillas filled with melted cheese, served with a mild salsa.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   4. PEANUT BUTTER AND BANANA SANDWICH   ●   ●   ●   Description: Peanut butter and banana slices on whole wheat bread.   Allergens: Nuts (peanut), Gluten.   Suitable for Vegetarians: Yes (if using vegetarian peanut butter)   5. VEGGIE PITA POCKETS   ●   ●   ●   Description: Mini whole wheat pita pockets filled with hummus, cucumber, and cherry tomatoes.   Allergens: Glu

## 4. Create the Agent for Amazon Bedrock

Let's go through all the steps to create an Agent for Amazon Bedrock. These are the steps to complete:
    
1. Create an Amazon DynamoDB table
2. Create an AWS Lambda function
3. Create the IAM policies needed for the Agent
4. Create the Agent
5. Create the Agent Action Group
6. Allow the Agent to invoke the Action Group Lambda
7. Associate the Knowledge Base to the agent
8. Prepare the Agent and create an alias

### 4.1 Create the DynamoDB table
Create a DynamoDB table which contains the restaurant bookings information.

In [13]:
table_name = 'restaurant_bookings'
create_dynamodb(table_name)

Creating table restaurant_bookings...
Table restaurant_bookings created successfully!


### 4.2 Create the Lambda Function

Create a lambda function that interacts with DynamoDB table. To do so we will:

1. Create the `lambda_function.py` file which contains the logic for our lambda function
2. Create the IAM role for our Lambda function
3. Create the lambda function with the required permissions

#### Create the function code
When creating an Agent for Amazon Bedrock, you can connect a Lambda function to the Action Group to execute the functions required by the agent. In this option, the agent is responsible for the execution of the functions. Let's create the lambda function that implements the functions for `get_booking_details`, `create_booking` and `delete_booking`

In [14]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_bookings')

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_booking_details(booking_id):
    """
    Retrieve details of a restaurant booking
    
    Args:
        booking_id (string): The ID of the booking to retrieve
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}


def create_booking(date, name, hour, num_guests):
    """
    Create a new restaurant booking
    
    Args:
        date (string): The date of the booking
        name (string): Name to idenfity your reservation
        hour (string): The hour of the booking
        num_guests (integer): The number of guests for the booking
    """
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        return {'error': str(e)}


def delete_booking(booking_id):
    """
    Delete an existing restaurant booking
    
    Args:
        booking_id (str): The ID of the booking to delete
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}
    

def lambda_handler(event, context):
    # get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')
    
    # name of the function that should be invoked
    function = event.get('function', '')
    
    # parameters to invoke function with
    parameters = event.get('parameters', [])

    if function == 'get_booking_details':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    elif function == 'create_booking':
        date = get_named_parameter(event, "date")
        name = get_named_parameter(event, "name")
        hour = get_named_parameter(event, "hour")
        num_guests = get_named_parameter(event, "num_guests")

        if date and hour and num_guests:
            response = str(create_booking(date, name, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'delete_booking':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Writing lambda_function.py


#### Create the required permissions
Now create the lambda role and its required policies. For this case, we need the lambda to be able to access DynamoDB, that is why we also create a DynamoDB policy and attach to our Lambda. To do so, we will use the support function `create_lambda_role`.

In [15]:
lambda_iam_role = create_lambda_role(agent_name, table_name)

#### Create the function

Now that we have the Lambda function code and its execution role, let's package it into a Zip file and create the Lambda resources

In [16]:
lambda_function_name = f'{agent_name}-lambda'

In [17]:
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### 4.3 Create the IAM policies needed for the Agent

Now that we have created the Knowledge Base, our DynamoDB table and the Lambda function to execute the tasks for our agent, let's start creating our Agent.


First need to create the agent policies that allow bedrock model invocation and Knowledge Base query and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model. Here we use the `create_agent_role_and_policies` to create the agent role and its required policies

In [18]:
agent_role = create_agent_role_and_policies(agent_name, agent_foundation_model, kb_id=kb_id)

In [19]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_booking-agent',
  'RoleId': 'AROAVBMQ6OXWSPWGY2VGN',
  'Arn': 'arn:aws:iam::346585724397:role/AmazonBedrockExecutionRoleForAgents_booking-agent',
  'CreateDate': datetime.datetime(2024, 7, 10, 14, 11, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': '17df26dc-ec61-4bde-997e-a66d97d7558e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Jul 2024 14:10:59 GMT',
   'x-amzn-requestid': '17df26dc-ec61-4bde-997e-a66d97d7558e',
   'content-type': 'text/xml',
   'content-length': '853'},
  'RetryAttempts': 0}}

### 4.4 Create the Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) api from boto3. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [20]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': 'bf13fd0b-9822-4405-95ea-90cf4ad844ed',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 10 Jul 2024 14:11:26 GMT',
   'content-type': 'application/json',
   'content-length': '665',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bf13fd0b-9822-4405-95ea-90cf4ad844ed',
   'x-amz-apigw-id': 'asx2PFOSPHcEivw=',
   'x-amzn-trace-id': 'Root=1-668e968e-674ee8f45aed5ad965955045'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:346585724397:agent/DIDXKHVZES',
  'agentId': 'DIDXKHVZES',
  'agentName': 'booking-agent',
  'agentResourceRoleArn': 'arn:aws:iam::346585724397:role/AmazonBedrockExecutionRoleForAgents_booking-agent',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 7, 10, 14, 11, 26, 157371, tzinfo=tzlocal()),
  'description': 'Agent in charge of a restaurants table bookings',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'instruction': 

Let's get our Agent ID. It will be important to perform operations with our agent

In [21]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

The agent id is: DIDXKHVZES


### 4.5 Create the Agent Action Group
We will now create an agent action group that uses the lambda function created before. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In this example, we will provide the Action Group functionality using a [`functionSchema`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html).

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [22]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

We now use the function schema to create the agent action group using the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) API

In [23]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [24]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '05de4ee4-df15-4d01-bec4-f277d31fb6fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Jul 2024 14:12:24 GMT',
   'content-type': 'application/json',
   'content-length': '1346',
   'connection': 'keep-alive',
   'x-amzn-requestid': '05de4ee4-df15-4d01-bec4-f277d31fb6fe',
   'x-amz-apigw-id': 'asx_aE-2vHcESrQ=',
   'x-amzn-trace-id': 'Root=1-668e96c8-6e8174d41ceb6b592386733e'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:346585724397:function:booking-agent-lambda'},
  'actionGroupId': 'QO8SLL6GOV',
  'actionGroupName': 'TableBookingsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'DIDXKHVZES',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 7, 10, 14, 12, 24, 878156, tzinfo=tzlocal()),
  'description': '\nActions for getting table booking information, create a new booking or delete an existing booking',
  'functionSchema': {'functions': [{'descripti

### 4.6 Allow the Agent to invoke the Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda). Let's add the resource-based policy to the lambda function created

In [25]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [26]:
response

{'ResponseMetadata': {'RequestId': '0d239430-a5c1-4e99-b84f-f5d3b829987f',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Wed, 10 Jul 2024 14:13:12 GMT',
   'content-type': 'application/json',
   'content-length': '348',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0d239430-a5c1-4e99-b84f-f5d3b829987f'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:346585724397:function:booking-agent-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:346585724397:agent/DIDXKHVZES"}}}'}

### 4.7 Associate the Knowledge Base to the agent
Now we have created the Agent we can go ahead and associate the Knowledge Base we created earlier. 

In [27]:
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about the plates in the menu.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

In [28]:
response

{'ResponseMetadata': {'RequestId': 'f9932cfd-bc30-41e5-9f38-5ed26ae2cc62',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 10 Jul 2024 14:13:20 GMT',
   'content-type': 'application/json',
   'content-length': '267',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f9932cfd-bc30-41e5-9f38-5ed26ae2cc62',
   'x-amz-apigw-id': 'asyIEHMGPHcEpkw=',
   'x-amzn-trace-id': 'Root=1-668e9700-42711d6b2ed924bd69d5058c'},
  'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2024, 7, 10, 14, 13, 20, 259457, tzinfo=tzlocal()),
  'description': 'Access the knowledge base when customers ask about the plates in the menu.',
  'knowledgeBaseId': 'MKEZTICEKV',
  'knowledgeBaseState': 'ENABLED',
  'updatedAt': datetime.datetime(2024, 7, 10, 14, 13, 20, 259457, tzinfo=tzlocal())}}

### 4.8 Prepare the Agent and create an alias

Let's create a DRAFT version of the agent that can be used for internal testing.


In [29]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': '38ed5afe-f98e-4e00-bb77-64b0df0e730f', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 10 Jul 2024 14:13:28 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '38ed5afe-f98e-4e00-bb77-64b0df0e730f', 'x-amz-apigw-id': 'asyJXGKcvHcECww=', 'x-amzn-trace-id': 'Root=1-668e9708-6383320d6edc6a5c42d2da72'}, 'RetryAttempts': 0}, 'agentId': 'DIDXKHVZES', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 7, 10, 14, 13, 28, 582752, tzinfo=tzlocal())}


You can invoke the DRAFT version of your agent using the test alias id `TSTALIASID` or you can create a new alias and a new version for your agent. Here we are also going to create an Agent alias to later on use to invoke it with the alias id created

In [30]:
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='TestAlias',
    agentId=agent_id,
    description='Test alias',
)

alias_id = response["agentAlias"]["agentAliasId"]
print("The Agent alias is:",alias_id)
time.sleep(30)

The Agent alias is: JOBKZ7LFTQ


## 5. Test the Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks. You can invoke your agent with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API

In [31]:
def invokeAgent(query, session_id, enable_trace=False, session_state=dict()):
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

##### Invoke Agent to query Knowledge Base
Let's now use our support `invokeAgent` function to query our Knowledge Base with the Agent

In [32]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What are the starters in the childrens menu?"
response = invokeAgent(query, session_id)
print(response)

Based on the menu information, the children's menu at our restaurant does not seem to have a dedicated "starters" section. However, some of the entree options could potentially serve as starters or appetizers for children, such as:

- Mini Cheese Quesadillas
- Veggie Pita Pockets 
- Peanut Butter and Banana Sandwich

<sources>
5
</sources>

CPU times: user 24.5 ms, sys: 0 ns, total: 24.5 ms
Wall time: 12.2 s


##### Invoke Agent to execute function from Action Group
Now let's test our Action Group functionality and create a new reservation

In [33]:
%%time
query = "Hi, I am Anna. I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
response = invokeAgent(query, session_id)
print(response)

Thank you Anna, I have successfully created a booking for 2 people at 8pm on May 5th, 2024. Your booking ID is d477c04f. Please let me know if you need anything else regarding this reservation.
CPU times: user 23.3 ms, sys: 0 ns, total: 23.3 ms
Wall time: 8.4 s


##### Invoke Agent with prompt attribute

We've used our agent to do the first reservation. However, often when booking tables in restaurants we are already logged in to systesm that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [`promptSessionAttributes`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [34]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invokeAgent(query, session_id, session_state=session_state)
print(response)

Your booking for 2 people on May 5th, 2024 at 8pm has been created successfully. Your booking ID is 23fd97ab.
CPU times: user 23.5 ms, sys: 0 ns, total: 23.5 ms
Wall time: 10.5 s


##### Validating prompt attribute
Let's now use our session context to validate that the reservation was done under the correct name

In [35]:
%%time
query = "What was the name used in my last reservation?"
response = invokeAgent(query, session_id)
print(response)

The name used for your booking on May 5th, 2024 at 8pm for 2 people was John.
CPU times: user 20.1 ms, sys: 3.86 ms, total: 24 ms
Wall time: 7.58 s


##### Retrieving information from the database in a new session

Next, let's confirm that our reservation system is working correctly. To do so, let's use our previous booking ID and retrieve our reservation details using a new session id


**Important**: remember to replace the information in next queries with your generated booking id

In [36]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to get the information for booking 007659d1"
response = invokeAgent(query, session_id)
print(response)

Sorry, I could not find any booking details for the ID 007659d1. It seems this booking ID does not exist in our system. Please double check the booking ID and try again.
CPU times: user 20.3 ms, sys: 0 ns, total: 20.3 ms
Wall time: 9.21 s


##### Canceling reservation

As plans change, we would now like to cancel the reservation we just did using our Agent for it.

In [37]:
%%time
query = "I want to delete the booking 007659d1"
response = invokeAgent(query, session_id)
print(response)

I'm afraid I could not find any booking with the ID 007659d1 in our system when you asked for the details earlier. Are you sure you want to delete this booking ID? If so, I can try deleting it, but it likely does not exist and the deletion will fail.
CPU times: user 25 ms, sys: 122 µs, total: 25.1 ms
Wall time: 7.23 s


And let's make sure everything worked out correctly

In [38]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to get the information for booking 007659d1"
response = invokeAgent(query, session_id)
print(response)

Sorry, I could not find any booking details for the ID 007659d1. It seems this booking ID does not exist in our system. Please double check the booking ID and try again.
CPU times: user 20.9 ms, sys: 3.03 ms, total: 23.9 ms
Wall time: 14.5 s


##### Handling context with PromptAttributes

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Agents for Amazon Bedrock also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [39]:
# retrieving today
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

'Jul-10-2024'

In [40]:
%%time
# reserving a table for tomorrow
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm tomorrow."
session_state = {
    "promptSessionAttributes": {
        "name": "John",
        "today": today
    }
}
response = invokeAgent(query, session_id, session_state=session_state)
print(response)

Your booking for 2 people at 8pm on Jul-11-2024 has been created successfully. Your booking ID is 8a94f9e1.
CPU times: user 17.5 ms, sys: 6.84 ms, total: 24.4 ms
Wall time: 7.84 s


And finally, let's validate our reservation

**Important**: remember to replace the booking id with the new one

In [41]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to get the information for booking 98e6464f"
response = invokeAgent(query, session_id)
print(response)

Sorry, there is no booking found with the ID 98e6464f. Please double check the booking ID you provided and try again.
CPU times: user 22.8 ms, sys: 1.44 ms, total: 24.3 ms
Wall time: 10.1 s


##### Invoke Agent with Trace

Agents for Amazon Bedrock also provides you with the details of steps being orchestrated by the Agent using the [Trace](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html). You can enable the trace during agent invocation. Let's now invoke the agent with the trace enabled

In [42]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the desserts on the adult menu?"
response = invokeAgent(query, session_id, enable_trace=True)
print(response)

[2024-07-10 14:30:53,723] p73 {2277358814.py:16} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 10 Jul 2024 14:30:53 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '03475440-3ec9-11ef-abcd-761a4312bd7b',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '68209e2e-ed1e-4466-ac3c-95a76a60f6ad'},
                      'HTTPStatusCode': 200,
                      'RequestId': '68209e2e-ed1e-4466-ac3c-95a76a60f6ad',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f0a9409f880>,
 'contentType': 'application/json',
 'sessionId': '03475440-3ec9-11ef-abcd-761a4312bd7b'}


[2024-07-10 14:30:53,996] p73 {2277358814.py:31} INFO - {
  "agentAliasId": "JOBKZ7LFTQ",
  "agentId": "DIDXKHVZES",
  "agentVersion": "1",
  "sessionId": "03475440-3ec9-11ef-abcd-761a4312bd7b",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\"        You are a restaurant agent, helping clients retrieve information from their booking, create a new booking or delete an existing booking        You have been provided with a set of functions to answer the user's question.        You must call the functions in the format below:        <function_calls>        <invoke>            <tool_name>$TOOL_NAME</tool_name>            <parameters>            <$PARAMETER_NAME>$PARAMETE


The desserts on the adult menu at The Regrettable Experience include:

- Classic New York Cheesecake with a graham cracker crust, topped with fruit compote or chocolate ganache (contains dairy and gluten)
- Apple Pie a la Mode with vanilla ice cream and caramel sauce (contains dairy and gluten)
- Chocolate Lava Cake with a molten center, served with raspberry sorbet (contains dairy and possible gluten/soy)
- Pecan Pie Bars with a buttery shortbread crust and pecan filling (contains dairy, nuts, and gluten)
- Banana Pudding Parfait with layers of vanilla pudding, bananas, vanilla wafers, whipped cream, and crushed nuts (contains dairy and gluten)

<sources>
3
</sources>

CPU times: user 36.9 ms, sys: 7.1 ms, total: 44 ms
Wall time: 21.5 s


## 6. Clean-up 
Let's delete all the associated resources created to avoid unnecessary costs. 

In [43]:
clean_up_resources(
    table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions, 
    agent_id, kb_id, alias_id
)

Agent DIDXKHVZES, Agent Alias JOBKZ7LFTQ, and Action Group have been deleted.
Lambda function booking-agent-lambda has been deleted.
Table restaurant_bookings is being deleted...
Table restaurant_bookings has been deleted.


In [44]:
# Delete the agent roles and policies
delete_agent_roles_and_policies(agent_name)

In [45]:
# delete KB
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

[2024-07-10 14:33:25,054] p73 {base.py:258} INFO - DELETE https://ltrrfl22o4hpqb5fz0e5.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-3465 [status:200 request:0.213s]
